In [ ]:
!pip install torch torchvision torchaudio


In [ ]:
!pip install xlsxwriter
!pip install darts


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.7/169.7 kB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:

In [ ]:
!pip install u8darts
!pip install pytorch-lightning


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from darts import TimeSeries
from darts.models import TiDEModel
from darts.dataprocessing.transformers import Scaler
import numpy as np

# Load data
supply_df = pd.read_excel("supply_data.xlsx")
demand_df = pd.read_excel("demand_data.xlsx")
covariate_df = pd.read_excel("covariate_data.xlsx")

# Prepare categorical encodings
def prepare_categorical_encodings(df):
    location_mapping = {loc: idx for idx, loc in enumerate(sorted(df['location'].unique()))}
    item_mapping = {item: idx for idx, item in enumerate(sorted(df['food_type'].unique()))}
    return {
        'location': [location_mapping[loc] for loc in df['location']],
        'food_type': [item_mapping[item] for item in df['food_type']]
    }

# Convert categorical columns to numerical values
supply_encoded = prepare_categorical_encodings(supply_df)
demand_encoded = prepare_categorical_encodings(demand_df)

# Create consolidated dataframes with proper date handling
supply_series_data = pd.DataFrame({
    **supply_encoded,
    'quantity': supply_df['quantity'],
    'date': pd.to_datetime(supply_df['date'])  # Direct date conversion
})

demand_series_data = pd.DataFrame({
    **demand_encoded,
    'quantity': demand_df['quantity'],
    'date': pd.to_datetime(demand_df['date'])  # Direct date conversion
})

# Handle duplicate dates by aggregating quantities
supply_series_data = supply_series_data.groupby(
    ['date', 'location', 'food_type']
)['quantity'].sum().reset_index()

demand_series_data = demand_series_data.groupby(
    ['date', 'location', 'food_type']
)['quantity'].sum().reset_index()

# Prepare covariate data
covariate_df['date'] = pd.to_datetime(covariate_df['date'])

# Pivot data to wide format
# After handling duplicates and datetime conversion...

# Pivot and flatten supply data
supply_pivot = supply_series_data.pivot_table(
    index='date',
    columns=['location', 'food_type'],
    values='quantity',
    aggfunc='sum'
)
supply_pivot.columns = [f"{loc}_{ft}" for loc, ft in supply_pivot.columns]
supply_pivot = supply_pivot.reset_index()

# Pivot and flatten demand data
demand_pivot = demand_series_data.pivot_table(
    index='date',
    columns=['location', 'food_type'],
    values='quantity',
    aggfunc='sum'
)
demand_pivot.columns = [f"{loc}_{ft}" for loc, ft in demand_pivot.columns]
demand_pivot = demand_pivot.reset_index()

supply_columns = supply_pivot.columns[1:].tolist()  # Skip 'date' column
demand_columns = demand_pivot.columns[1:].tolist()  # Skip 'date' column

# Create TimeSeries objects with string column names
ts_supply = TimeSeries.from_dataframe(
    supply_pivot,
    time_col='date',
    value_cols=supply_columns
)

ts_demand = TimeSeries.from_dataframe(
    demand_pivot,
    time_col='date',
    value_cols=demand_columns
)
# Covariates remain the same
ts_covariates = TimeSeries.from_dataframe(
    covariate_df,
    time_col='date',
    value_cols=['temperature', 'holiday']
)

# Train-test split
split_idx = int(0.8 * len(ts_supply))
train_supply, test_supply = ts_supply[:split_idx], ts_supply[split_idx:]
train_demand, test_demand = ts_demand[:split_idx], ts_demand[split_idx:]
train_covariates = ts_covariates[:split_idx]

# Data scaling
scaler_supply = Scaler()
train_supply_scaled = scaler_supply.fit_transform(train_supply)
scaler_demand = Scaler()
train_demand_scaled = scaler_demand.fit_transform(train_demand)

# Model setup and training
model_supply = TiDEModel(
    input_chunk_length=30,
    output_chunk_length=7,
    n_epochs=10
)

model_demand = TiDEModel(
    input_chunk_length=30,
    output_chunk_length=7,
    n_epochs=10
)

model_supply.fit(train_supply_scaled, past_covariates=train_covariates)
model_demand.fit(train_demand_scaled, past_covariates=train_covariates)

# Save models
model_supply.save("supply_model.pkl")
model_demand.save("demand_model.pkl")

# Prediction
forecast_supply = scaler_supply.inverse_transform(model_supply.predict(
    n=7,
    past_covariates=ts_covariates
))

forecast_demand = scaler_demand.inverse_transform(model_demand.predict(
    n=7,
    past_covariates=ts_covariates
))

# MAPE calculation with zero handling
def safe_mape(actual_series, pred_series):
    actual_values = actual_series.values()
    pred_values = pred_series.values()

    # Handle zero values
    non_zero_mask = actual_values != 0
    if not np.any(non_zero_mask):
        return float('nan')

    safe_actual = np.where(non_zero_mask, actual_values, np.nan)
    safe_actual = np.nan_to_num(safe_actual, nan=np.nanmean(safe_actual))

    return 100 * np.mean(np.abs((safe_actual - pred_values) / safe_actual))

# Evaluation
print(f"Supply MAPE: {safe_mape(test_supply[:7], forecast_supply):.2f}%")
print(f"Demand MAPE: {safe_mape(test_demand[:7], forecast_demand):.2f}%")


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | criterion           | MSELoss          | 0      | train
1 | train_criterion     | MSELoss          | 0      | train
2 | val_criterion       | MSELoss          | 0      | train
3 | train_metrics       | MetricCollection | 0      | train
4 | val_metrics         | MetricCollection | 0      | train
5 | past_cov_projection | _ResidualBlock   | 912    | train
6 | encoders            | Sequential       | 853 K  | train
7 | decoders            | Sequential       | 45.4 K | train
8 | temporal_decod

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | criterion           | MSELoss          | 0      | train
1 | train_criterion     | MSELoss          | 0      | train
2 | val_criterion       | MSELoss          | 0      | train
3 | train_metrics       | MetricCollection | 0      | train
4 | val_metrics         | MetricCollection | 0      | train
5 | past_cov_projection | _ResidualBlock   | 912    | train
6 | encoders            | Sequential       | 85

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Supply MAPE: nan%
Demand MAPE: nan%


In [ ]:
# Save models
model_supply.save("supply_model.pkl")
model_demand.save("demand_model.pkl")

# Verify files exist
import os
print(os.path.exists("supply_model.pkl"))  # Should print True
print(os.path.exists("demand_model.pkl"))  # Should print True

True
True
